In [1]:
import torch
import torchvision
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import math

In [2]:
data_path = "/home/anirudh/ml/cv/dataset/dogs-vs-cats-redux-kernels-edition/train"

img_size = 48

def load_images_from_folder(folder):
    count = 0
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img_rs = cv2.resize(img, (img_size, img_size), 3)
        if img is not None:
            images.append(img_rs.astype("float32")/255.0)
            if("dog" in filename):
                labels.append(0)
            elif("cat" in filename):
                labels.append(1)
        count+=1
        if(count%200==0):
            print(count)
    return images, labels



In [ ]:

images, labels = load_images_from_folder(data_path)

In [3]:
class CatsDogsDataset(Dataset):
    def __init__(self):
        self.x = torch.tensor(images)
        self.y = torch.tensor(np.array(labels).astype('float32'))
        self.y = self.y.view(self.y.shape[0],1)
        self.n_samples = len(labels)
    
    def __getitem__(self, index):
        return self.x[index].view(3,48,48), self.y[index]
    
    def __len__(self):
        return self.n_samples
    
train_dataset = CatsDogsDataset()
    

NameError: name 'images' is not defined

In [4]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 9 * 9, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)

    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 22, 22 
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 16, 9, 9
        x = x.view(-1, 16 * 9 * 9)            # -> n, 1296           
        x = F.relu(self.fc1(x))               # -> n, 120
        x = F.relu(self.fc2(x))               # -> n, 84
        x = self.fc3(x)                       # -> n, 1
        x = torch.sigmoid(x)                  
        return x

In [ ]:
batch_size = 128
num_epochs = 25
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True)
model = ConvNet()
criterion = nn.BCELoss()
learning_rate = 0.001

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        with torch.no_grad():
            y_predicted_cls = outputs.round()
            acc  = y_predicted_cls.eq(labels).sum()/float(labels.shape[0])

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 80 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}, Accuracy: {acc}')

print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)



In [6]:
test_data_path = "/home/anirudh/ml/cv/dataset/dogs-vs-cats-redux-kernels-edition/test"
test_images, labels = load_images_from_folder(test_data_path)

200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800
10000
10200
10400
10600
10800
11000
11200
11400
11600
11800
12000
12200
12400


In [5]:
model = ConvNet()
FILE = './cnn.pth'
model.load_state_dict(torch.load(FILE))
model.eval()

ConvNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=1296, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=1, bias=True)
)

In [ ]:


test_images = torch.tensor(test_images)
test_images = test_images.view(12500, 3, 48, 48)
import pandas as pd
output_list = []

y = model(test_images)

In [ ]:

for i in range(len(y)):
    output_list.append({'id':i+1, 'label': 1-y[i].item()})
df = pd.DataFrame(output_list)
df.to_csv('output.csv')
    

In [ ]:
predict(test_images)   